In [ ]:
# Row 0 = species labels (e.g., "Chinook", "Sockeye", "Coho", etc.)
# Row 1 = data types (e.g., "2022.0", "10-Year Daily", etc.)
url = "https://www.nws.usace.army.mil/Missions/Civil-Works/Locks-and-Dams/Chittenden-Locks/Fish-Counts/"


def open_ballard_locks_data(file_path, yyyy):

    # Load multi-header Excel
    df_raw = pd.read_excel(
        file_path, sheet_name="Historical", header=[0, 1], skiprows=[2]
    )

    # Fix first column name
    cols = list(df_raw.columns)
    cols[0] = ("Date", "Date")
    df_raw.columns = pd.MultiIndex.from_tuples(cols)

    # Drop empty columns
    df_raw = df_raw.dropna(axis=1, how="all")

    # Clean Date column
    df_raw[("Date", "Date")] = pd.to_datetime(df_raw[("Date", "Date")], errors="coerce")
    df_raw = df_raw[df_raw[("Date", "Date")].notna()]

    # Keep the Date column separate
    df_dates = df_raw[("Date", "Date")]

    # Select all columns except Date for melting
    df_data = df_raw.drop(columns=[("Date", "Date")])

    # Stack the first level of columns (species), turning species into index level
    df_long = df_data.stack(level=0, future_stack=True)  # stack species level

    # Reset index to get Date, species and the inner columns as columns
    df_long = df_long.reset_index()

    # Rename columns: level_0 is original index (row), level_1 is species
    df_long = df_long.rename(
        columns={
            "level_0": "row_index",  # or drop later if not needed
            "level_1": "Species",
        }
    )

    # Join the Date back
    df_long["Date"] = df_dates.values[df_long["row_index"]]

    # Drop the row_index if you want
    df_long = df_long.drop(columns=["row_index"])

    # Filter to Columns
    df_long = df_long[["Species", "Date", yyyy]]
    df_long = df_long.rename(columns={yyyy: "Daily_Count"})

    return df_long

In [ ]:
ballard_data = [
    (
        "/Users/tylerstevenson/Documents/CODE/SalmonSignal/data/BallardLocks 09182022.xlsx",
        2022,
    ),
    (
        "/Users/tylerstevenson/Documents/CODE/SalmonSignal/data/Fish count 07102023.xlsx",
        2023,
    ),
    (
        "/Users/tylerstevenson/Documents/CODE/SalmonSignal/data/Fish count 10022024.xlsx",
        2024,
    ),
    (
        "/Users/tylerstevenson/Documents/CODE/SalmonSignal/data/Fish count 07012025.xlsx",
        2025,
    ),
]

df_combine = []
for item in ballard_data:
    df_long = open_ballard_locks_data(file_path=item[0], yyyy=item[1])
    df_combine.append(df_long)

df_combine = pd.concat(df_combine)
df_combine["Year"] = df_combine["Date"].dt.year
df_combine["DoY"] = df_combine["Date"].dt.day_of_year

chinook_ = df_combine[df_combine.Species == "Chinook"]

fig = px.line(chinook_, x="DoY", y="Daily_Count", color="Year")
fig.show()